(Baseline_Feature_Transformation)=
# Transformación de características de referencia

El conjunto de datos simulado generado en la sección anterior es simple. Solo contiene las características esenciales que caracterizan una transacción con tarjeta de pago. Estas son: un identificador único para la transacción, la fecha y hora de la transacción, el monto de la transacción, un identificador único para el cliente, un número único para el comerciante y una variable binaria que etiqueta la transacción como legítima o fraudulenta (0 para legítima o 1 para fraudulenta). La Fig. 1 proporciona las primeras tres filas del conjunto de datos simulado:
 
![alt text](images/tx_table.png)
<p style="text-align: center;">
Fig. 1. Las primeras tres transacciones en el conjunto de datos simulado utilizado en este capítulo.
</p>



Lo que cada fila resume esencialmente es que, a las 00:00:31, el 1 de abril de 2018, un cliente con ID 596 realizó un pago por un valor de 57.19 a un comerciante con ID 3156, y que la transacción no fue fraudulenta. Luego, a las 00:02:10, el 1 de abril de 2018, un cliente con ID 4961 realizó un pago por un valor de 81.51 a un comerciante con ID 3412, y que la transacción no fue fraudulenta. Y así sucesivamente. El conjunto de datos simulado es una larga lista de tales transacciones (1.8 millones en total). La variable `transaction_ID` es un identificador único para cada transacción.  

Si bien es conceptualmente simple para un humano, tal conjunto de características no es apropiado para un modelo predictivo de aprendizaje automático. Los algoritmos de aprendizaje automático suelen requerir características *numéricas* y *ordenadas*. Numérico significa que el tipo de la variable debe ser un número entero o un número real. Ordenado significa que el orden de los valores de una variable es significativo. 

En este conjunto de datos, las únicas características numéricas y ordenadas son el monto de la transacción y la etiqueta de fraude. La fecha es una marca de tiempo de Panda y, por lo tanto, no es numérica. Los identificadores de las transacciones, clientes y terminales son numéricos pero no ordenados: no tendría sentido asumir, por ejemplo, que el terminal con ID 3548 es 'más grande' o 'mayor' que el terminal con ID 1983. Más bien, estos identificadores representan 'entidades' distintas, a las que se hace referencia como características *categóricas*. 

Desafortunadamente, no existe un procedimiento estándar para tratar con características no numéricas o categóricas. El tema se conoce en la literatura de aprendizaje automático como *ingeniería de características* o *transformación de características*. En esencia, el objetivo de la ingeniería de características es diseñar nuevas características que se asume que son relevantes para un problema predictivo. El diseño de estas características suele depender del problema e implica conocimiento del dominio.

En esta sección, implementaremos tres tipos de transformación de características que se sabe que son relevantes para la detección de fraude con tarjetas de pago.

![encoding](images/encoding_variables.png)

El primer tipo de transformación involucra la variable de fecha/hora y consiste en crear características binarias que caracterizan períodos potencialmente relevantes. Crearemos dos características de este tipo. La primera caracterizará si una transacción ocurre durante un día laborable o durante el fin de semana. La segunda caracterizará si una transacción ocurre durante el día o la noche. Estas características pueden ser útiles ya que se ha observado en conjuntos de datos del mundo real que los patrones fraudulentos difieren entre días laborables y fines de semana, y entre el día y la noche.  

El segundo tipo de transformación involucra la identificación del cliente y consiste en crear características que caracterizan los comportamientos de gasto del cliente. Seguiremos el marco RFM (Recencia, Frecuencia, Valor Monetario) propuesto en {cite}`VANVLASSELAER201538`, y realizaremos un seguimiento del monto promedio de gasto y el número de transacciones para cada cliente y para tres tamaños de ventana. Esto conducirá a la creación de seis nuevas características.

El tercer tipo de transformación involucra la identificación del terminal y consiste en crear nuevas características que caracterizan el 'riesgo' asociado con el terminal. El riesgo se definirá como el número promedio de fraudes que se observaron en el terminal para tres tamaños de ventana. Esto conducirá a la creación de tres nuevas características. 

La siguiente tabla resume los tipos de transformación que se realizarán y las nuevas características que se crearán. 

|Nombre de característica original|Tipo de característica original|Transformación|Número de nuevas características|Tipo de nueva(s) característica(s)|
|---|---|---|---|---|
|TX\_DATE\_TIME | Marca de tiempo Panda |0 si la transacción es durante un día laborable, 1 si es durante un fin de semana. La nueva característica se llama TX_DURING_WEEKEND.|1|Entero (0/1)|
|TX\_DATE\_TIME | Marca de tiempo Panda |0 si la transacción es entre las 6 am y las 0 pm, 1 si es entre las 0 pm y las 6 am. La nueva característica se llama TX_DURING_NIGHT.|1|Entero (0/1)|
|CUSTOMER\_ID | Variable categórica |Número de transacciones por el cliente en los últimos n día(s), para n en {1,7,30}. Las nuevas características se llaman CUSTOMER_ID_NB_TX_nDAY_WINDOW.|3|Entero|
|CUSTOMER\_ID | Variable categórica |Monto promedio de gasto en los últimos n día(s), para n en {1,7,30}. Las nuevas características se llaman CUSTOMER_ID_AVG_AMOUNT_nDAY_WINDOW.|3|Real|
|TERMINAL\_ID | Variable categórica |Número de transacciones en el terminal en los últimos n+d día(s), para n en {1,7,30} y d=7. El parámetro d se llama retraso y se discutirá más adelante en este cuaderno. Las nuevas características se llaman TERMINAL_ID_NB_TX_nDAY_WINDOW.|3|Entero|
|TERMINAL\_ID | Variable categórica |Número promedio de fraudes en el terminal en los últimos n+d día(s), para n en {1,7,30} y d=7. El parámetro d se llama retraso y se discutirá más adelante en este cuaderno. Las nuevas características se llaman TERMINAL_ID_RISK_nDAY_WINDOW.|3|Real|

Las siguientes secciones proporcionan la implementación para cada una de estas tres transformaciones. Después de las transformaciones, se creará un conjunto de 14 nuevas características. Tenga en cuenta que algunas de las características son el resultado de funciones de agregación sobre los valores de otras características o condiciones (mismo cliente, ventana de tiempo dada). Estas características a menudo se denominan *características agregadas*.

In [24]:
# Inicialización: Cargar funciones compartidas y datos simulados 

# Cargar funciones compartidas
!curl -O https://raw.githubusercontent.com/Fraud-Detection-Handbook/fraud-detection-handbook/main/Chapter_References/shared_functions.py
%run shared_functions.py

# Obtener datos simulados del repositorio de Github
if not os.path.exists("simulated-data-raw"):
    !git clone https://github.com/Fraud-Detection-Handbook/simulated-data-raw
        

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 31567  100 31567    0     0   135k      0 --:--:-- --:--:-- --:--:--  135k
Cloning into 'simulated-data-raw'...
remote: Enumerating objects: 189, done.
remote: Counting objects: 100% (189/189), done.
remote: Compressing objects: 100% (187/187), done.
remote: Total 189 (delta 0), reused 186 (delta 0), pack-reused 0
Receiving objects: 100% (189/189), 28.04 MiB | 3.13 MiB/s, done.


## Carga del conjunto de datos

Primero carguemos los datos de transacciones simulados en el cuaderno anterior. Cargaremos los archivos de transacciones de abril a septiembre. Los archivos se pueden cargar utilizando la función `read_from_files` en el cuaderno [funciones compartidas](shared_functions). La función se puso en este cuaderno ya que se utilizará con frecuencia a lo largo de este libro.

La función toma como entrada la carpeta donde se encuentran los archivos de datos y las fechas que definen el período a cargar (entre `BEGIN_DATE` y `END_DATE`). Devuelve un DataFrame de transacciones. Las transacciones se ordenan por orden cronológico. 


In [3]:
DIR_INPUT='./simulated-data-raw/data/' 

BEGIN_DATE = "2018-04-01"
END_DATE = "2018-09-30"

print("Cargar archivos")
%time transactions_df=read_from_files(DIR_INPUT, BEGIN_DATE, END_DATE)
print("{0} transacciones cargadas, conteniendo {1} transacciones fraudulentas".format(len(transactions_df),transactions_df.TX_FRAUD.sum()))


Load  files
CPU times: user 3.1 s, sys: 696 ms, total: 3.79 s
Wall time: 4.13 s
1754155 transactions loaded, containing 14681 fraudulent transactions


In [4]:
transactions_df.head()

,TRANSACTION_ID,TX_DATETIME,CUSTOMER_ID,TERMINAL_ID,TX_AMOUNT,TX_TIME_SECONDS,TX_TIME_DAYS,TX_FRAUD,TX_FRAUD_SCENARIO
0,0,2018-04-01 00:00:31,596,3156,57.16,31,0,0,0
1,1,2018-04-01 00:02:10,4961,3412,81.51,130,0,0,0
2,2,2018-04-01 00:07:56,2,1365,146.00,476,0,0,0
3,3,2018-04-01 00:09:29,4128,8737,64.49,569,0,0,0
4,4,2018-04-01 00:10:34,927,9906,50.99,634,0,0,0


## Transformaciones de fecha y hora

Crearemos dos nuevas características binarias a partir de las fechas y horas de las transacciones:

* La primera caracterizará si una transacción ocurre durante un día laborable (valor 0) o un fin de semana (1), y se llamará `TX_DURING_WEEKEND`
* La segunda caracterizará si una transacción ocurre durante el día (0) o durante la noche (1). La noche se define como las horas que están entre las 0 pm y las 6 am. Se llamará `TX_DURING_NIGHT`. 

Para la característica `TX_DURING_WEEKEND`, definimos una función `is_weekend` que toma como entrada una marca de tiempo de Panda y devuelve 1 si la fecha es durante un fin de semana, o 0 en caso contrario. El objeto de marca de tiempo proporciona convenientemente la función `weekday` para ayudar a calcular este valor.

In [5]:
def is_weekend(tx_datetime):
    
    # Transformar fecha en día de la semana (0 es lunes, 6 es domingo)
    weekday = tx_datetime.weekday()
    # Valor binario: 0 si es día laborable, 1 si es fin de semana
    is_weekend = weekday>=5
    
    return int(is_weekend)


Es entonces sencillo calcular esta característica para todas las transacciones utilizando la función `apply` de Panda. 

In [6]:
%time transactions_df['TX_DURING_WEEKEND']=transactions_df.TX_DATETIME.apply(is_weekend)

CPU times: user 7.54 s, sys: 247 ms, total: 7.79 s
Wall time: 7.94 s


Seguimos la misma lógica para implementar la característica `TX_DURING_NIGHT`. Primero, una función `is_night` que toma como entrada una marca de tiempo de Panda y devuelve 1 si la hora es durante la noche, o 0 en caso contrario. El objeto de marca de tiempo proporciona convenientemente la propiedad `hour` para ayudar a calcular este valor.

In [7]:
def is_night(tx_datetime):
    
    # Obtener la hora de la transacción
    tx_hour = tx_datetime.hour
    # Valor binario: 1 si la hora es menor que 6, y 0 en caso contrario
    is_night = tx_hour<=6
    
    return int(is_night)

In [8]:
%time transactions_df['TX_DURING_NIGHT']=transactions_df.TX_DATETIME.apply(is_night)

CPU times: user 7.09 s, sys: 221 ms, total: 7.31 s
Wall time: 7.47 s


Verifiquemos que estas características se calcularon correctamente.

In [9]:
transactions_df[transactions_df.TX_TIME_DAYS>=30]

,TRANSACTION_ID,TX_DATETIME,CUSTOMER_ID,TERMINAL_ID,TX_AMOUNT,TX_TIME_SECONDS,TX_TIME_DAYS,TX_FRAUD,TX_FRAUD_SCENARIO,TX_DURING_WEEKEND,TX_DURING_NIGHT
288062,288062,2018-05-01 00:01:21,3546,2944,18.71,2592081,30,0,0,0,1
288063,288063,2018-05-01 00:01:48,206,3521,18.60,2592108,30,0,0,0,1
288064,288064,2018-05-01 00:02:22,2610,4470,66.67,2592142,30,0,0,0,1
288065,288065,2018-05-01 00:03:15,4578,1520,79.41,2592195,30,0,0,0,1
288066,288066,2018-05-01 00:03:51,1246,7809,52.08,2592231,30,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...
1754150,1754150,2018-09-30 23:56:36,161,655,54.24,15810996,182,0,0,1,0
1754151,1754151,2018-09-30 23:57:38,4342,6181,1.23,15811058,182,0,0,1,0
1754152,1754152,2018-09-30 23:58:21,618,1502,6.62,15811101,182,0,0,1,0
1754153,1754153,2018-09-30 23:59:52,4056,3067,55.40,15811192,182,0,0,1,0


El 2018-05-01 fue un lunes, y el 2018-09-30 un domingo. Estas fechas están correctamente marcadas como día laborable y fin de semana, respectivamente. La característica de día y noche también se establece correctamente para las primeras transacciones, que ocurren poco después de las 0 pm, y las últimas transacciones que ocurren poco antes de las 0 pm. 

## Customer ID transformations

Procedamos ahora con las transformaciones de ID de cliente. Nos inspiraremos en el marco RFM (Recencia, Frecuencia, Valor Monetario) propuesto en {cite}`VANVLASSELAER201538`, y calcularemos dos de estas características durante tres ventanas de tiempo. La primera característica será el número de transacciones que ocurren dentro de una ventana de tiempo (Frecuencia). La segunda será el monto promedio gastado en estas transacciones (Valor Monetario). Las ventanas de tiempo se establecerán en uno, siete y treinta días. Esto generará seis nuevas características. Tenga en cuenta que estas ventanas de tiempo podrían optimizarse más adelante junto con los modelos utilizando un procedimiento de selección de modelos ([Capítulo 5](Model_Selection)). 

Implementemos estas transformaciones escribiendo una función `get_customer_spending_behaviour_features`. La función toma como entrada el conjunto de transacciones para un cliente y un conjunto de tamaños de ventana. Devuelve un DataFrame con las seis nuevas características. Nuestra implementación se basa en la función `rolling` de Panda, que facilita el cálculo de agregados durante una ventana de tiempo.


In [10]:
def get_customer_spending_behaviour_features(customer_transactions, windows_size_in_days=[1,7,30]):
    
    # Primero ordenemos las transacciones cronológicamente
    customer_transactions=customer_transactions.sort_values('TX_DATETIME')
    
    # La fecha y hora de la transacción se establece como el índice, lo que permitirá el uso de la función rolling
    customer_transactions.index=customer_transactions.TX_DATETIME
    
    # Para cada tamaño de ventana
    for window_size in windows_size_in_days:
        
        # Calcular la suma de los montos de transacción y el número de transacciones para el tamaño de ventana dado
        SUM_AMOUNT_TX_WINDOW=customer_transactions['TX_AMOUNT'].rolling(str(window_size)+'d').sum()
        NB_TX_WINDOW=customer_transactions['TX_AMOUNT'].rolling(str(window_size)+'d').count()
    
        # Calcular el monto promedio de transacción para el tamaño de ventana dado
        # NB_TX_WINDOW siempre es >0 ya que la transacción actual siempre está incluida
        AVG_AMOUNT_TX_WINDOW=SUM_AMOUNT_TX_WINDOW/NB_TX_WINDOW
    
        # Guardar valores de características
        customer_transactions['CUSTOMER_ID_NB_TX_'+str(window_size)+'DAY_WINDOW']=list(NB_TX_WINDOW)
        customer_transactions['CUSTOMER_ID_AVG_AMOUNT_'+str(window_size)+'DAY_WINDOW']=list(AVG_AMOUNT_TX_WINDOW)
    
    # Reindexar según los ID de transacción
    customer_transactions.index=customer_transactions.TRANSACTION_ID
        
    # Y devolver el dataframe con las nuevas características
    return customer_transactions


Calculemos estos agregados para el primer cliente.

In [11]:
spending_behaviour_customer_0=get_customer_spending_behaviour_features(transactions_df[transactions_df.CUSTOMER_ID==0])
spending_behaviour_customer_0

,TRANSACTION_ID,TX_DATETIME,CUSTOMER_ID,TERMINAL_ID,TX_AMOUNT,TX_TIME_SECONDS,TX_TIME_DAYS,TX_FRAUD,TX_FRAUD_SCENARIO,TX_DURING_WEEKEND,TX_DURING_NIGHT,CUSTOMER_ID_NB_TX_1DAY_WINDOW,CUSTOMER_ID_AVG_AMOUNT_1DAY_WINDOW,CUSTOMER_ID_NB_TX_7DAY_WINDOW,CUSTOMER_ID_AVG_AMOUNT_7DAY_WINDOW,CUSTOMER_ID_NB_TX_30DAY_WINDOW,CUSTOMER_ID_AVG_AMOUNT_30DAY_WINDOW
TRANSACTION_ID,,,,,,,,,,,,,,,,,
1758,1758,2018-04-01 07:19:05,0,6076,123.59,26345,0,0,0,1,0,1.0,123.590000,1.0,123.590000,1.0,123.590000
8275,8275,2018-04-01 18:00:16,0,858,77.34,64816,0,0,0,1,0,2.0,100.465000,2.0,100.465000,2.0,100.465000
8640,8640,2018-04-01 19:02:02,0,6698,46.51,68522,0,0,0,1,0,3.0,82.480000,3.0,82.480000,3.0,82.480000
12169,12169,2018-04-02 08:51:06,0,6569,54.72,118266,1,0,0,0,0,3.0,59.523333,4.0,75.540000,4.0,75.540000
15764,15764,2018-04-02 14:05:38,0,7707,63.30,137138,1,0,0,0,0,4.0,60.467500,5.0,73.092000,5.0,73.092000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1750390,1750390,2018-09-30 13:38:41,0,3096,38.23,15773921,182,0,0,1,0,5.0,64.388000,28.0,57.306429,89.0,63.097640
1750758,1750758,2018-09-30 14:10:21,0,9441,43.60,15775821,182,0,0,1,0,6.0,60.923333,29.0,56.833793,89.0,62.433933
1751039,1751039,2018-09-30 14:34:30,0,1138,69.69,15777270,182,0,0,1,0,7.0,62.175714,29.0,57.872414,90.0,62.514556


Podemos verificar que las nuevas características son consistentes con el perfil del cliente (ver el cuaderno anterior). Para el cliente 0, el monto medio fue `mean_amount`=62.26, y la frecuencia de transacción fue `mean_nb_tx_per_day`=2.18. Estos valores coinciden estrechamente con las características `CUSTOMER_ID_NB_TX_30DAY_WINDOW` y `CUSTOMER_ID_AVG_AMOUNT_30DAY_WINDOW`, especialmente después de 30 días.

Generemos ahora estas características para todos los clientes. Esto es sencillo utilizando los métodos `groupby` y `apply` de Panda.

In [12]:
%time transactions_df=transactions_df.groupby('CUSTOMER_ID').apply(lambda x: get_customer_spending_behaviour_features(x, windows_size_in_days=[1,7,30]))
transactions_df=transactions_df.sort_values('TX_DATETIME').reset_index(drop=True)


CPU times: user 1min 2s, sys: 1.21 s, total: 1min 3s
Wall time: 1min 7s


In [13]:
transactions_df

,TRANSACTION_ID,TX_DATETIME,CUSTOMER_ID,TERMINAL_ID,TX_AMOUNT,TX_TIME_SECONDS,TX_TIME_DAYS,TX_FRAUD,TX_FRAUD_SCENARIO,TX_DURING_WEEKEND,TX_DURING_NIGHT,CUSTOMER_ID_NB_TX_1DAY_WINDOW,CUSTOMER_ID_AVG_AMOUNT_1DAY_WINDOW,CUSTOMER_ID_NB_TX_7DAY_WINDOW,CUSTOMER_ID_AVG_AMOUNT_7DAY_WINDOW,CUSTOMER_ID_NB_TX_30DAY_WINDOW,CUSTOMER_ID_AVG_AMOUNT_30DAY_WINDOW
0,0,2018-04-01 00:00:31,596,3156,57.16,31,0,0,0,1,1,1.0,57.160000,1.0,57.160000,1.0,57.160000
1,1,2018-04-01 00:02:10,4961,3412,81.51,130,0,0,0,1,1,1.0,81.510000,1.0,81.510000,1.0,81.510000
2,2,2018-04-01 00:07:56,2,1365,146.00,476,0,0,0,1,1,1.0,146.000000,1.0,146.000000,1.0,146.000000
3,3,2018-04-01 00:09:29,4128,8737,64.49,569,0,0,0,1,1,1.0,64.490000,1.0,64.490000,1.0,64.490000
4,4,2018-04-01 00:10:34,927,9906,50.99,634,0,0,0,1,1,1.0,50.990000,1.0,50.990000,1.0,50.990000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1754150,1754150,2018-09-30 23:56:36,161,655,54.24,15810996,182,0,0,1,0,2.0,75.280000,12.0,67.047500,72.0,69.521111
1754151,1754151,2018-09-30 23:57:38,4342,6181,1.23,15811058,182,0,0,1,0,1.0,1.230000,21.0,22.173810,93.0,24.780753
1754152,1754152,2018-09-30 23:58:21,618,1502,6.62,15811101,182,0,0,1,0,5.0,7.368000,21.0,7.400476,65.0,7.864462
1754153,1754153,2018-09-30 23:59:52,4056,3067,55.40,15811192,182,0,0,1,0,3.0,100.696667,16.0,107.052500,51.0,102.919608


## Transformaciones de ID de terminal

Finalmente, procedamos con las transformaciones de ID de terminal. El objetivo principal será extraer un *puntaje de riesgo*, que evalúa la exposición de un ID de terminal dado a transacciones fraudulentas. El puntaje de riesgo se definirá como el número promedio de transacciones fraudulentas que ocurrieron en un ID de terminal durante una ventana de tiempo. Al igual que con las transformaciones de ID de cliente, utilizaremos tres tamaños de ventana, de 1, 7 y 30 días.

A diferencia de las transformaciones de ID de cliente, las ventanas de tiempo no precederán directamente a una transacción dada. En su lugar, se desplazarán hacia atrás por un *período de retraso*. El período de retraso explica el hecho de que, en la práctica, las transacciones fraudulentas solo se descubren después de una investigación de fraude o una queja del cliente. Por lo tanto, las etiquetas fraudulentas, que son necesarias para calcular el puntaje de riesgo, solo están disponibles después de este período de retraso. Como primera aproximación, este período de retraso se establecerá en una semana. Las motivaciones para el período de retraso se argumentarán más adelante en el [Capítulo 5, Estrategias de validación](Validation_Strategies). 

Realicemos el cálculo de los puntajes de riesgo definiendo una función `get_count_risk_rolling_window`. La función toma como entrada el DataFrame de transacciones para un ID de terminal dado, el período de retraso y una lista de tamaños de ventana. En la primera etapa, el número de transacciones y transacciones fraudulentas se calculan para el período de retraso (`NB_TX_DELAY` y `NB_FRAUD_DELAY`). En la segunda etapa, el número de transacciones y transacciones fraudulentas se calculan para cada tamaño de ventana más el período de retraso (`NB_TX_DELAY_WINDOW` y `NB_FRAUD_DELAY_WINDOW`). El número de transacciones y transacciones fraudulentas que ocurrieron para un tamaño de ventana dado, desplazado hacia atrás por el período de retraso, se obtiene simplemente calculando las diferencias de las cantidades obtenidas para el período de retraso y el tamaño de ventana más el período de retraso:

```
NB_FRAUD_WINDOW=NB_FRAUD_DELAY_WINDOW-NB_FRAUD_DELAY
NB_TX_WINDOW=NB_TX_DELAY_WINDOW-NB_TX_DELAY
```

El puntaje de riesgo se obtiene finalmente calculando la proporción de transacciones fraudulentas para cada tamaño de ventana (o 0 si no ocurrió ninguna transacción para la ventana dada):

```
RISK_WINDOW=NB_FRAUD_WINDOW/NB_TX_WINDOW
```

Además del puntaje de riesgo, la función también devuelve el número de transacciones para cada tamaño de ventana. Esto resulta en la adición de seis nuevas características: El riesgo y el número de transacciones, para tres tamaños de ventana.


In [14]:
def get_count_risk_rolling_window(terminal_transactions, delay_period=7, windows_size_in_days=[1,7,30], feature="TERMINAL_ID"):
    
    terminal_transactions=terminal_transactions.sort_values('TX_DATETIME')
    
    terminal_transactions.index=terminal_transactions.TX_DATETIME
    
    NB_FRAUD_DELAY=terminal_transactions['TX_FRAUD'].rolling(str(delay_period)+'d').sum()
    NB_TX_DELAY=terminal_transactions['TX_FRAUD'].rolling(str(delay_period)+'d').count()
    
    for window_size in windows_size_in_days:
    
        NB_FRAUD_DELAY_WINDOW=terminal_transactions['TX_FRAUD'].rolling(str(delay_period+window_size)+'d').sum()
        NB_TX_DELAY_WINDOW=terminal_transactions['TX_FRAUD'].rolling(str(delay_period+window_size)+'d').count()
    
        NB_FRAUD_WINDOW=NB_FRAUD_DELAY_WINDOW-NB_FRAUD_DELAY
        NB_TX_WINDOW=NB_TX_DELAY_WINDOW-NB_TX_DELAY
    
        RISK_WINDOW=NB_FRAUD_WINDOW/NB_TX_WINDOW
        
        terminal_transactions[feature+'_NB_TX_'+str(window_size)+'DAY_WINDOW']=list(NB_TX_WINDOW)
        terminal_transactions[feature+'_RISK_'+str(window_size)+'DAY_WINDOW']=list(RISK_WINDOW)
        
    terminal_transactions.index=terminal_transactions.TRANSACTION_ID
    
    # Reemplazar valores NA con 0 (todos los puntajes de riesgo indefinidos donde NB_TX_WINDOW es 0) 
    terminal_transactions.fillna(0,inplace=True)
    
    return terminal_transactions


In [15]:
transactions_df[transactions_df.TX_FRAUD==1]

,TRANSACTION_ID,TX_DATETIME,CUSTOMER_ID,TERMINAL_ID,TX_AMOUNT,TX_TIME_SECONDS,TX_TIME_DAYS,TX_FRAUD,TX_FRAUD_SCENARIO,TX_DURING_WEEKEND,TX_DURING_NIGHT,CUSTOMER_ID_NB_TX_1DAY_WINDOW,CUSTOMER_ID_AVG_AMOUNT_1DAY_WINDOW,CUSTOMER_ID_NB_TX_7DAY_WINDOW,CUSTOMER_ID_AVG_AMOUNT_7DAY_WINDOW,CUSTOMER_ID_NB_TX_30DAY_WINDOW,CUSTOMER_ID_AVG_AMOUNT_30DAY_WINDOW
3527,3527,2018-04-01 10:17:43,3774,3059,225.41,37063,0,1,1,1,0,3.0,158.073333,3.0,158.073333,3.0,158.073333
5789,5790,2018-04-01 13:31:48,4944,6050,222.26,48708,0,1,1,1,0,2.0,127.605000,2.0,127.605000,2.0,127.605000
6549,6549,2018-04-01 14:42:02,4625,9102,226.40,52922,0,1,1,1,0,4.0,167.165000,4.0,167.165000,4.0,167.165000
9583,9583,2018-04-02 01:01:05,3814,6893,59.15,90065,1,1,3,0,1,6.0,29.138333,6.0,29.138333,6.0,29.138333
10356,10355,2018-04-02 05:03:35,2513,1143,222.04,104615,1,1,1,0,1,5.0,123.740000,5.0,123.740000,5.0,123.740000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1753524,1753524,2018-09-30 19:51:48,1671,3192,128.60,15796308,182,1,3,1,0,6.0,138.358333,25.0,106.957200,82.0,75.621341
1753600,1753600,2018-09-30 20:09:00,4166,632,17.39,15797340,182,1,2,1,0,3.0,19.766667,19.0,15.984737,86.0,15.846512
1753673,1753673,2018-09-30 20:30:52,4097,1558,24.04,15798652,182,1,2,1,0,3.0,23.050000,16.0,40.440625,63.0,41.877460
1754014,1754014,2018-09-30 22:27:04,100,8604,73.85,15805624,182,1,3,1,0,2.0,48.010000,26.0,30.384231,103.0,23.627184


Calculemos estas seis características para el primer ID de terminal que contiene al menos un fraude:

In [16]:
# Obtener el primer ID de terminal que contiene fraudes
transactions_df[transactions_df.TX_FRAUD==0].TERMINAL_ID[0]

3156

In [17]:
get_count_risk_rolling_window(transactions_df[transactions_df.TERMINAL_ID==3059], delay_period=7, windows_size_in_days=[1,7,30])

,TRANSACTION_ID,TX_DATETIME,CUSTOMER_ID,TERMINAL_ID,TX_AMOUNT,TX_TIME_SECONDS,TX_TIME_DAYS,TX_FRAUD,TX_FRAUD_SCENARIO,TX_DURING_WEEKEND,...,CUSTOMER_ID_NB_TX_7DAY_WINDOW,CUSTOMER_ID_AVG_AMOUNT_7DAY_WINDOW,CUSTOMER_ID_NB_TX_30DAY_WINDOW,CUSTOMER_ID_AVG_AMOUNT_30DAY_WINDOW,TERMINAL_ID_NB_TX_1DAY_WINDOW,TERMINAL_ID_RISK_1DAY_WINDOW,TERMINAL_ID_NB_TX_7DAY_WINDOW,TERMINAL_ID_RISK_7DAY_WINDOW,TERMINAL_ID_NB_TX_30DAY_WINDOW,TERMINAL_ID_RISK_30DAY_WINDOW
TRANSACTION_ID,,,,,,,,,,,,,,,,,,,,,
3527,3527,2018-04-01 10:17:43,3774,3059,225.41,37063,0,1,1,1,...,3.0,158.073333,3.0,158.073333,0.0,0.0,0.0,0.0,0.0,0.0
4732,4732,2018-04-01 11:59:14,55,3059,36.28,43154,0,0,0,1,...,2.0,35.670000,2.0,35.670000,0.0,0.0,0.0,0.0,0.0,0.0
16216,16216,2018-04-02 14:47:34,4879,3059,105.00,139654,1,0,0,0,...,10.0,76.010000,10.0,76.010000,0.0,0.0,0.0,0.0,0.0,0.0
18249,18249,2018-04-02 19:08:10,2263,3059,90.89,155290,1,0,0,0,...,7.0,50.458571,7.0,50.458571,0.0,0.0,0.0,0.0,0.0,0.0
26512,26512,2018-04-03 15:44:49,4879,3059,58.51,229489,2,0,0,0,...,14.0,71.070000,14.0,71.070000,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1697944,1697944,2018-09-25 05:32:56,402,3059,57.30,15312776,177,0,0,0,...,14.0,65.167857,46.0,68.163261,1.0,0.0,9.0,0.0,36.0,0.0
1701971,1701971,2018-09-25 12:30:54,1035,3059,7.56,15337854,177,0,0,0,...,23.0,7.052174,107.0,6.763738,2.0,0.0,10.0,0.0,36.0,0.0
1704512,1704512,2018-09-25 16:37:41,1519,3059,35.79,15352661,177,0,0,0,...,7.0,41.404286,30.0,46.780000,1.0,0.0,9.0,0.0,36.0,0.0


Podemos verificar que el primer fraude ocurrió el 2018/09/10, y que los puntajes de riesgo solo comienzan a contarse con una semana de retraso. 

Generemos finalmente estas características para todos los terminales. Esto es sencillo utilizando los métodos `groupby` y `apply` de Panda. 

In [18]:
%time transactions_df=transactions_df.groupby('TERMINAL_ID').apply(lambda x: get_count_risk_rolling_window(x, delay_period=7, windows_size_in_days=[1,7,30], feature="TERMINAL_ID"))
transactions_df=transactions_df.sort_values('TX_DATETIME').reset_index(drop=True)


CPU times: user 2min 27s, sys: 2.23 s, total: 2min 29s
Wall time: 2min 41s


In [19]:
transactions_df

,TRANSACTION_ID,TX_DATETIME,CUSTOMER_ID,TERMINAL_ID,TX_AMOUNT,TX_TIME_SECONDS,TX_TIME_DAYS,TX_FRAUD,TX_FRAUD_SCENARIO,TX_DURING_WEEKEND,...,CUSTOMER_ID_NB_TX_7DAY_WINDOW,CUSTOMER_ID_AVG_AMOUNT_7DAY_WINDOW,CUSTOMER_ID_NB_TX_30DAY_WINDOW,CUSTOMER_ID_AVG_AMOUNT_30DAY_WINDOW,TERMINAL_ID_NB_TX_1DAY_WINDOW,TERMINAL_ID_RISK_1DAY_WINDOW,TERMINAL_ID_NB_TX_7DAY_WINDOW,TERMINAL_ID_RISK_7DAY_WINDOW,TERMINAL_ID_NB_TX_30DAY_WINDOW,TERMINAL_ID_RISK_30DAY_WINDOW
0,0,2018-04-01 00:00:31,596,3156,57.16,31,0,0,0,1,...,1.0,57.160000,1.0,57.160000,0.0,0.0,0.0,0.0,0.0,0.00000
1,1,2018-04-01 00:02:10,4961,3412,81.51,130,0,0,0,1,...,1.0,81.510000,1.0,81.510000,0.0,0.0,0.0,0.0,0.0,0.00000
2,2,2018-04-01 00:07:56,2,1365,146.00,476,0,0,0,1,...,1.0,146.000000,1.0,146.000000,0.0,0.0,0.0,0.0,0.0,0.00000
3,3,2018-04-01 00:09:29,4128,8737,64.49,569,0,0,0,1,...,1.0,64.490000,1.0,64.490000,0.0,0.0,0.0,0.0,0.0,0.00000
4,4,2018-04-01 00:10:34,927,9906,50.99,634,0,0,0,1,...,1.0,50.990000,1.0,50.990000,0.0,0.0,0.0,0.0,0.0,0.00000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1754150,1754150,2018-09-30 23:56:36,161,655,54.24,15810996,182,0,0,1,...,12.0,67.047500,72.0,69.521111,1.0,0.0,4.0,0.0,28.0,0.00000
1754151,1754151,2018-09-30 23:57:38,4342,6181,1.23,15811058,182,0,0,1,...,21.0,22.173810,93.0,24.780753,1.0,0.0,9.0,0.0,39.0,0.00000
1754152,1754152,2018-09-30 23:58:21,618,1502,6.62,15811101,182,0,0,1,...,21.0,7.400476,65.0,7.864462,1.0,0.0,5.0,0.0,33.0,0.00000
1754153,1754153,2018-09-30 23:59:52,4056,3067,55.40,15811192,182,0,0,1,...,16.0,107.052500,51.0,102.919608,1.0,0.0,6.0,0.0,28.0,0.00000


## Guardado del conjunto de datos

Finalmente guardemos el conjunto de datos, dividido en lotes diarios, utilizando el formato pickle. 

In [22]:
DIR_OUTPUT = "./simulated-data-transformed/"

if not os.path.exists(DIR_OUTPUT):
    os.makedirs(DIR_OUTPUT)

start_date = datetime.datetime.strptime("2018-04-01", "%Y-%m-%d")

for day in range(transactions_df.TX_TIME_DAYS.max()+1):
    
    transactions_day = transactions_df[transactions_df.TX_TIME_DAYS==day].sort_values('TX_TIME_SECONDS')
    
    date = start_date + datetime.timedelta(days=day)
    filename_output = date.strftime("%Y-%m-%d")+'.pkl'
    
    # Protocol=4 requerido para Google Colab
    transactions_day.to_pickle(DIR_OUTPUT+filename_output, protocol=4)

El conjunto de datos generado también está disponible en Github en `https://github.com/Fraud-Detection-Handbook/simulated-data-transformed`.